In [1]:
from collections import Counter
import pandas as pd
from datasets import load_dataset
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

# EDA TyDi QA
Notebook untuk memahami dataset TyDi QA

In [59]:
tydiqa_gold = load_dataset("khalidalt/tydiqa-goldp", 'indonesian', trust_remote_code=True)
tydiqa_gold

DatasetDict({
    train: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 5702
    })
    validation: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 565
    })
})

In [62]:
tydiqa_gold['train'][0]['answers']

{'text': ['28 Agustus 1950'], 'start_byte': [45], 'limit_byte': [60]}

In [9]:
# Ambil kolom 'question_text' dari dataset train
questions = tydiqa_gold['train']['question_text']

# Ambil kata pertama dari setiap pertanyaan
first_words = [question.lower().split()[0] if question.strip() else None for question in questions]

# Hitung jumlah kemunculan tiap kata pertama
first_word_counts = Counter(first_words)

# Konversi hasil ke DataFrame untuk tampilan lebih mudah
first_word_df = pd.DataFrame(first_word_counts.items(), columns=["First Word", "Count"]).sort_values(by="Count", ascending=False)

# Tampilkan hasil
first_word_df.head(10)

,First Word,Count
5,apakah,953
2,kapan,919
6,apa,851
4,siapakah,528
7,berapakah,406
8,berapa,391
1,siapa,379
3,dimana,255
0,dimanakah,245
9,kapankah,238


In [ ]:
## Beberapa contoh row dimana jawaban ground truth lumayan panjang
long_answers_id = ['8223462044041100059-0', '8101241799056890701-0', '-1836421090681989358-7']

## Menggunakan meta-llama_Llama-3.2-1B-Instruct-50_results.json sbg Proxy

*Jika malas berurusan dengan library datasets*

In [5]:
tydi_with_generated = pd.read_json("meta-llama_Llama-3.2-1B-Instruct-50_results.json")

### Mismatch antara jenis pertanyaan (dimana) dengan jenis jawaban (waktu)

In [10]:
# Definisikan kata-kata pertama yang ingin dicari
target_words = ['Dimana', 'dimanakah', 'Dimanakah', 'Dimankah', 'dimankah']

# Ambil subset data di mana question_text diawali dengan salah satu kata di target_words
filtered_data = [
    {"id": qid, "question_text": question, "answers": answers}
    for qid, question, answers in zip(tydiqa_gold['train']['id'], tydiqa_gold['train']['question_text'], tydiqa_gold['train']['answers'])
    if question.split()[0] in target_words
]

# Konversi ke DataFrame untuk tampilan lebih mudah
filtered_df = pd.DataFrame(filtered_data)

In [15]:

# List ID yang ingin dicari
target_ids = [
    '496955121539262633-33', '-2609773390563272077-2', '-946255727274322126-4', 
    '-7749839855121808364-0', '4442256442092795257-8', '199208750651388504-2', 
    '6542172453366557069-7', '4919195816976139224-0', '1443260107176351473-1', 
    '6273515731581201009-9', '-7557435993194156840-18', '-5596213448756892216-7',  
    '-515183342052641630-0', '-4344805552053190465-2', '-1356951969628235591-3', 
    '132761720256571903-0', '733277053417590077-0', '8422091779176670015-4'
]

# Filter baris di mana kolom 'id' ada dalam target_ids
wrong_where_q = filtered_df[filtered_df['id'].isin(target_ids)]
num_wrong_where_q = wrong_where_q.shape[0]
print("Jumlah row yg pertanyaan 'Dimana' malah dijawab dengan jawaban waktu:", num_wrong_where_q)

percent_of_wrong_where_q = num_wrong_where_q * 100 / filtered_df.shape[0]
print("Persentase terhadap seluruh pertanyaan 'Dimana':", percent_of_wrong_where_q)

wrong_where_q.head()

Jumlah row yg pertanyaan 'Dimana' malah dijawab dengan jawaban waktu: 18
Persentase terhadap seluruh pertanyaan 'Dimana': 3.5502958579881656


,id,question_text,answers
0,496955121539262633-33,dimanakah Dr. Ernest François Eugène Douwes Dekker meninggal?,"{'text': ['28 Agustus 1950'], 'start_byte': [45], 'limit_byte': [60]}"
3,-2609773390563272077-2,Dimanakah Kucing Pallas pertama kali ditemukan ?,"{'text': ['2008'], 'start_byte': [961], 'limit_byte': [965]}"
24,-946255727274322126-4,"dimanakah Joseph Patrick ""Joe"" Kennedy dilahirkan?","{'text': ['1858'], 'start_byte': [318], 'limit_byte': [322]}"
28,-7749839855121808364-0,Dimanakah produksi film pertama dilakukan ?,"{'text': ['abad 20'], 'start_byte': [434], 'limit_byte': [441]}"
33,4442256442092795257-8,dimanakah Fiber optik pertama kali diciptakan?,"{'text': ['1970'], 'start_byte': [2952], 'limit_byte': [2956]}"


# EDA Mr. TyDi

In [49]:
mr_tydi = load_dataset("castorini/mr-tydi", "indonesian")
mr_tydi

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 4902
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 1224
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages'],
        num_rows: 829
    })
})

In [58]:
from datasets import load_dataset

# Load dataset
mr_tydi = load_dataset("castorini/mr-tydi", "indonesian")

# Query yang dicari
target_query = 'dimanakah Joseph Patrick "Joe" Kennedy dilahirkan?'

# Variabel untuk menyimpan hasil
matching_rows = []

# Periksa setiap split dalam dataset
for split_name, split_data in mr_tydi.items():
    for row in split_data:
        if row["query"] == target_query:
            matching_rows.append({"split": split_name, **row})

# Tampilkan hasil
if matching_rows:
    print(f"Ditemukan {len(matching_rows)} baris dengan query yang cocok:")
    for row in matching_rows:
        print(row)
else:
    print("Tidak ditemukan baris dengan query yang cocok.")

Ditemukan 1 baris dengan query yang cocok:
{'split': 'train', 'query_id': '349', 'query': 'dimanakah Joseph Patrick "Joe" Kennedy dilahirkan?', 'positive_passages': [{'docid': '12980#4', 'text': 'John Fitzgerald Kennedy lahir di 83 Beals Street, Brookline, Massachusetts, pada tanggal 29 Mei 1917[10] dari pasangan pebisnis/politikus Joseph Patrick "Joe" Kennedy, Sr. (1888–1969) dan filantropis Rose Elizabeth Fitzgerald (1890–1995). Joe adalah putra sulung pebisnis/politikus Patrick Joseph "P. J." Kennedy (1858–1929) dan Mary Augusta Hickey (1857–1923). Rose adalah putri sulung Wali Kota Boston John Francis "Honey Fitz" Fitzgerald (1863–1950) dan Mary Josephine "Josie" Hannon (1865–1964). Keempat kakek-neneknya adalah anak-anak imigran Irlandia.[1]', 'title': 'John F. Kennedy'}], 'negative_passages': [{'docid': '203636#36', 'text': 'Karena nya filantropi dan persahabatan dekat dengan Francis Spellman, Uskup Agung New York (kemudian Kardinal), selama ini, Joseph Kennedy diinvestasikan seb

#### Memahami kolom dan nilai dari Mr TyDi

In [53]:
# Ambil 5 baris acak dari split 'train'
random_sample = mr_tydi['train'].shuffle(seed=42).select(range(5))

# Tampilkan isi baris yang diambil
for row in random_sample:
    print(row)
    print()

{'query_id': '1922', 'query': 'kapan Nokia berdiri ?', 'positive_passages': [{'docid': '26149#1', 'text': 'Kata Nokia berasal dari nama sebuah komunitas yang tinggal di sungai Emakoski di negara Finlandia Selatan. Nokia didirikan sebagai perusahaan penggilingan pulp oleh Fredrik Idestam pada tahun 1865. Perusahaan Karet Finlandia kemudian mendirikan pabriknya di kawasan sekitarnya pada awal abad ke-20 dan mulai menggunakan merek Nokia.', 'title': 'Nokia'}], 'negative_passages': [{'docid': '603582#0', 'text': 'Nokia N78 adalah produk telepon genggam yang dirilis oleh perusahaan Nokia. Telepon genggam ini memiliki dimensi 113 x 49 x 15.1 mm dengan berat 101.8 gram. Yang membuat ponsel ini terlihat futuristik adalah adanya penggabungan desain dari nokia Nokia N73 dan Nokia N77 menjadikan Nokia N78 menarik berkat desainer professional, Joeske Schellen.', 'title': 'Nokia N78'}, {'docid': '444657#0', 'text': 'Ovi by Nokia adalah merek layanan Internet Nokia yang dapat digunakan dari peranti 

__Memahami kolom positive_passages__

In [54]:
import json

# Ambil salah satu row dari random_sample
sample_row = random_sample[0]  # Mengambil baris pertama dari random_sample
with open("contoh_mr_tydi", "w") as f:
        json.dump(sample_row, f, indent=4)
# # Periksa isi kolom positive_passages
# positive_passages = sample_row['positive_passages']
# positive_passages

__Memahami kolom negative_passages__

In [11]:
# Periksa isi kolom positive_passages
negative_passages = sample_row['negative_passages']
print("jumlah negative passages dalam 1 row random:", len(negative_passages))
negative_passages[:5]

jumlah negative passages dalam 1 row random: 29


[{'docid': '603582#0',
  'text': 'Nokia N78 adalah produk telepon genggam yang dirilis oleh perusahaan Nokia. Telepon genggam ini memiliki dimensi 113 x 49 x 15.1 mm dengan berat 101.8 gram. Yang membuat ponsel ini terlihat futuristik adalah adanya penggabungan desain dari nokia Nokia N73 dan Nokia N77 menjadikan Nokia N78 menarik berkat desainer professional, Joeske Schellen.',
  'title': 'Nokia N78'},
 {'docid': '444657#0',
  'text': 'Ovi by Nokia adalah merek layanan Internet Nokia yang dapat digunakan dari peranti bergerak, komputer (melalui Nokia Ovi Suite) atau situs web (Ovi.com). Nokia berfokus pada lima bidang layanan kunci: permainan, peta, media, pengiriman pesan, dan musik. Ovi ditujukan untuk dapat menyertakan pihak ketiga, seperti operator dan layanan pihak ketiga seperti Flickr. Dengan diumumkannya API Ovi Maps Player , Nokia telah mulai mengembangkan layanan mereka menjadi suatu platform yang memungkinkan pihak ketiga untuk menggunakan layanan Ovi Nokia. Ovi memiliki pe

__Mengecek jumlah positive_passages terbanyak, ditemukan bahwa semua positive_passages berisi satu elemen saja__

In [12]:
# Tentukan split yang ingin diperiksa, misalnya 'train'
split = mr_tydi['train']

# Cari row dengan positive_passages terbanyak
max_pos_row = max(split, key=lambda row: len(row['positive_passages']))

# Tampilkan jumlah positive_passages dan isi row tersebut
print(f"Jumlah positive_passages terbanyak: {len(max_pos_row['positive_passages'])}")
print("Row dengan positive_passages terbanyak:")
print(max_pos_row)

Jumlah positive_passages terbanyak: 1
Row dengan positive_passages terbanyak:
{'query_id': '0', 'query': 'dimanakah  Dr. Ernest François Eugène Douwes Dekker meninggal?', 'positive_passages': [{'docid': '7080#33', 'text': 'Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.', 'title': 'Ernest Douwes Dekker'}], 'negative_passages': [{'docid': '7074#0', 'text': 'dr. Tjipto Mangoenkoesoemo (EYD: Cipto Mangunkusumo) (Pecangaan, Jepara, Jawa Tengah, 1886 – Jakarta, 8 Maret 1943) adalah seorang tokoh pergerakan kemerdekaan Indonesia. Bersama dengan Ernest Douwes Dekker dan Ki Hajar Dewantara ia dikenal sebagai "Tiga Serangkai" yang banyak menyebarluaskan ide pemerintahan sendiri dan kritis terhadap pemerintahan penjajahan Hindia Belanda. Ia adalah tokoh dalam Indische Partij, suatu organisasi politik yang pertama kali mencetuskan ide pemerintahan sendiri di tangan penduduk s

In [ ]:
# Tentukan split yang ingin diperiksa, misalnya 'train'
split = mr_tydi['train']

# Cari semua row yang memiliki positive_passages sama dengan 0
rows_with_zero_positive_passages = [row for row in split if len(row['positive_passages']) == 0]

# Periksa apakah ada row yang memenuhi kriteria
if rows_with_zero_positive_passages:
    print(f"Terdapat {len(rows_with_zero_positive_passages)} row dengan positive_passages bernilai 0.")
    # Tampilkan salah satu row untuk melihat contohnya
    print("Contoh row:", rows_with_zero_positive_passages[0])
else:
    print("Tidak ada row dengan positive_passages sama dengan 0.")

Tidak ada row dengan positive_passages sama dengan 0.


In [22]:
# Cari row dengan negative_passages terbanyak
max_neg_row = max(split, key=lambda row: len(row['negative_passages']))

# Tampilkan jumlah negative_passages terbanyak dan isi row tersebut
print(f"Jumlah negative_passages terbanyak: {len(max_neg_row['negative_passages'])}")
print("Row dengan negative_passages terbanyak:")
print(max_neg_row)
print()
# Kebalikannya: negative_passages tersedikit
min_neg_row = min(split, key=lambda row: len(row['negative_passages']))
print(f"Jumlah negative_passages tersedikit: {len(min_neg_row['negative_passages'])}")
print("Row dengan negative_passages tersedikit:")
print(min_neg_row)

Jumlah negative_passages terbanyak: 30
Row dengan negative_passages terbanyak:
{'query_id': '1', 'query': 'Siapa arsitek Balai Kota Seoul?', 'positive_passages': [{'docid': '2693620#5', 'text': 'Pada tanggal 18 Februari 2008, desain Yoo Kerl dari iArc terpilih dalam kompetisi desain Pemerintah Metropolitan Seoul.[6] Bagian atas depan bangunan baru ini dirancang untuk menonjol dalam bentuk lengkung. Perancang Yoo merangkul tiga kata kunci yakni "tradisi, warga negara, dan masa depan" dalam rancangannya, yang mencerminkan elemen horisontal dari arsitektur tradisional tingkat rendah Korea dan penafsiran ulang dari nuansa mendalam dan lekukan atap atap.[7]', 'title': 'Balai Kota Seoul'}], 'negative_passages': [{'docid': '2692513#0', 'text': 'Seoul Plaza adalah sebuah ruang terbuka umum yang berlokasi di depan Balai Kota Seoul di Taepyeongno, Jung-gu di Seoul, Korea Selatan. Plaza ini dibuka kembali pada 1 Mei 2004, oleh Pemerintah Metropolitan Seoul, dengan tujuan menyediakan ruang terbuka

In [20]:
# Tentukan split yang ingin diperiksa, misalnya 'train'
split = mr_tydi['train']

# Hitung total panjang negative_passages
total_negative_passages = sum(len(row['negative_passages']) for row in split)

# Hitung rata-rata
average_negative_passages = total_negative_passages / len(split)

# Cetak hasil
print(f"Rata-rata jumlah negative_passages: {average_negative_passages:.2f}")

Rata-rata jumlah negative_passages: 29.29


# Natural Questions (Versi RECOMP)

### 5 sampel saja

In [ ]:
# Path ke file JSON
file_path = "./data/extract-nq-train-ORIGINAL.json"

# Load dataset
nq_train_few = load_dataset("json", data_files=file_path)

# Tampilkan info dataset
print(nq_train_few)

DatasetDict({
    train: Dataset({
        features: ['question', 'em', 'f1', 'gold_answers', 'pred_answer', 'has_gold_answer', 'sentence', 'title', 'title+sentence', 'query', 'retrieved_docs', 'num_sentences', 'contriever_scores', 'lm_scores', 'dpr_instance'],
        num_rows: 5
    })
})


In [ ]:
sample = nq_train_few['train'][0]

## RECOMP - extractive

### Seluruh data

## RECOMP - Abstractive

In [1]:
from datasets import load_dataset

# Path ke file dataset
data_files = {
    "train": "./data/RECOMP-abstractive-nq-train-ORIGINAL.json",
    "dev": "./data/RECOMP-abstractive-nq-dev-ORIGINAL.json"
}

# Load dataset dengan menentukan split
abstract_nq = load_dataset("json", data_files=data_files)

# Tampilkan informasi dataset
print(abstract_nq)

DatasetDict({
    train: Dataset({
        features: ['question', 'has_gold_answer_in_prompt', 'gpt3_completions', 'gold_answers', 'generated_answers', 'gold_answer_score', 'gold_answer_with_max_score', 'pred_answer', 'em', 'f1', 'no_retrieval_em', 'no_retrieval_df', 'no_retrieval_pred_answer', 'passages', 'top_5_retrieval_em', 'top_5_retrieval_df', 'top_5_retrieval_pred_answer', 'type', 'summary'],
        num_rows: 39466
    })
    dev: Dataset({
        features: ['question', 'has_gold_answer_in_prompt', 'gpt3_completions', 'gold_answers', 'generated_answers', 'gold_answer_score', 'gold_answer_with_max_score', 'pred_answer', 'em', 'f1', 'no_retrieval_em', 'no_retrieval_df', 'no_retrieval_pred_answer', 'passages', 'top_5_retrieval_em', 'top_5_retrieval_df', 'top_5_retrieval_pred_answer', 'type', 'summary'],
        num_rows: 4931
    })
})


__kolom penting:__

`sudah dapat dari dataset` 
- question
- passages
- gold_answers

`perlu di-generate`
- gpt3_completions
- summary
- generated_answers
- gold_answers_score
- no_retrieval_pred_answer
- exact match & F1

In [7]:
abstract_nq['train'][1]['question']

'where will the 2022 fifa world cup be hosted'

In [4]:
abstract_nq['train'][1]['summary']

'The 2022 FIFA World Cup will be hosted by Qatar, the first time a World Cup will be held in the Arab World and the second time in Asia since 2002. The allocation of slots for each confederation was decided in 2015, and although there was a request to expand the number of teams from 32 to 48, the FIFA congress rejected the proposal.'

In [23]:
# answers = ["['2018']", "['Qatar']", "['to help, defend']", "['white', 'light blue']", "['Prince Edward Island', 'Milton, Ontario', 'the UK']", "['\"law of attraction\"']"]

False

##### Memahami distribusi data: bagaimana jika generated tidak/sama dengan gold answer dan bagaimana jika no_retrieval_pred tidak/sama dgn gold. 

In [28]:
import json
import ast

# Fungsi untuk memfilter kasus unik
def filter_cases(dataset):
    cases = {"case_1": [], "case_2": [], "case_3": [], "case_4": []}

    # Menentukan batas maksimum contoh per kasus
    max_examples = 3

    for row in dataset:
        if row["summary"] == " ":

            gold_answers_list = ast.literal_eval(row["gold_answers"])
            generated = row["generated_answers"]
            no_retrieval = row["no_retrieval_pred_answer"]

            # Case 1: Both generated_answers and no_retrieval_pred_answer match gold_answers
            if generated in gold_answers_list and no_retrieval in gold_answers_list:
                if len(cases["case_1"]) < max_examples:
                    cases["case_1"].append(row)
            # Case 2: Only generated_answers matches gold_answers
            elif generated in gold_answers_list and no_retrieval not in gold_answers_list:
                if len(cases["case_2"]) < max_examples:
                    cases["case_2"].append(row)
            # Case 3: Only no_retrieval_pred_answer matches gold_answers
            elif generated not in gold_answers_list and no_retrieval in gold_answers_list:
                if len(cases["case_3"]) < max_examples:
                    cases["case_3"].append(row)
            # Case 4: Neither matches gold_answers
            elif generated not in gold_answers_list and no_retrieval not in gold_answers_list:
                if len(cases["case_4"]) < max_examples:
                    cases["case_4"].append(row)

            # Jika semua kasus sudah memiliki cukup contoh, hentikan iterasi
            if all(len(cases[case]) >= max_examples for case in cases):
                break

    return cases

# Simpan hasil ke file JSON
def save_cases_to_json(cases, output_path):
    with open(output_path, "w") as f:
        json.dump(cases, f, indent=4)
    print(f"Hasil disimpan di: {output_path}")

# Filter kasus pada data training
sampled_cases_train = filter_cases(abstract_nq["train"])

# Simpan hasil ke file JSON
save_cases_to_json(sampled_cases_train, "inspect_empty_summary.json")


Hasil disimpan di: inspect_generated_no-retrieval-pred_gold.json


##### Memahami distribusi data ketika summary di-set ke kosong

In [3]:
from datasets import DatasetDict
import ast


# Fungsi untuk menghitung kasus
def count_cases(dataset):
    # Filter row dengan summary bernilai " "
    filtered_dataset = dataset.filter(lambda x: x["summary"] == " ")
    
    # Inisialisasi penghitung untuk masing-masing kasus
    case_1 = 0
    case_2 = 0
    case_3 = 0
    case_4 = 0

    for row in filtered_dataset:
        # Parse gold_answers menjadi list
        gold_answers_list = ast.literal_eval(row["gold_answers"])
        
        # Ambil generated_answers dan no_retrieval_pred_answer
        generated = row["generated_answers"]
        no_retrieval = row["no_retrieval_pred_answer"]

        # Tentukan kasusnya
        if generated in gold_answers_list and no_retrieval in gold_answers_list:
            case_1 += 1  # Baik generated dan no_retrieval sama dengan gold_answers
        elif generated in gold_answers_list and no_retrieval not in gold_answers_list:
            case_2 += 1  # Hanya generated yang sama dengan gold_answers
        elif generated not in gold_answers_list and no_retrieval in gold_answers_list:
            case_3 += 1  # Hanya no_retrieval yang sama dengan gold_answers
        elif generated not in gold_answers_list and no_retrieval not in gold_answers_list:
            case_4 += 1  # Keduanya tidak sama dengan gold_answers

    # Return hasil penghitungannya
    return {
        "case_1": case_1,
        "case_2": case_2,
        "case_3": case_3,
        "case_4": case_4
    }

# Hitung kasus pada data training
cases_train = count_cases(abstract_nq["train"])

# # Hitung kasus pada data dev
# cases_dev = count_cases(abstract_nq["dev"])

# Tampilkan hasil
print("Hasil pada data training:")
print(cases_train)
# print("\nHasil pada data dev:")
# print(cases_dev)


Filter:   0%|          | 0/39466 [00:00<?, ? examples/s]

Hasil pada data training:
{'case_1': 0, 'case_2': 0, 'case_3': 8986, 'case_4': 799}


##### Mengecek apakah generated_answers sama saja dengan pred_answer

In [4]:
# Fungsi untuk menghitung jumlah row dengan generated_answers != pred_answer
def count_generated_not_equal_pred(dataset):
    # Filter dataset berdasarkan kondisi
    filtered_dataset = dataset.filter(lambda x: x["generated_answers"] != x["pred_answer"])
    # Hitung jumlah row yang memenuhi kondisi
    return len(filtered_dataset)

# Hitung jumlah untuk data training
count_train = count_generated_not_equal_pred(abstract_nq["train"])

# Hitung jumlah untuk data dev
count_dev = count_generated_not_equal_pred(abstract_nq["dev"])

# Tampilkan hasil
print(f"Jumlah row dengan generated_answers != pred_answer pada data training: {count_train}")
# print(f"Jumlah row dengan generated_answers != pred_answer pada data dev: {count_dev}")


Filter:   0%|          | 0/39466 [00:00<?, ? examples/s]

Jumlah row dengan generated_answers != pred_answer pada data training: 3


In [6]:
import json

# Fungsi untuk memfilter row dengan generated_answers != pred_answer
def get_rows_generated_not_equal_pred(dataset):
    return dataset.filter(lambda x: x["generated_answers"] != x["pred_answer"])

# Ambil row yang sesuai dari split train
filtered_rows_train = get_rows_generated_not_equal_pred(abstract_nq["train"])

# Konversi ke dictionary
filtered_rows_dict = filtered_rows_train.to_dict()

# Simpan ke file JSON
output_file = "nq_abstract_where_pred-answer_not_equal_to_generated.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_rows_dict, f, indent=4)

print(f"{len(filtered_rows_train)} row berhasil disimpan ke {output_file}")

3 row berhasil disimpan ke nq_abstract_where_pred-answer_not_equal_to_generated.json


##### Menghitung jumlah row dimana pred_answer ataupun generated_answers kosong

In [8]:
# Fungsi untuk memfilter row dengan pred_answer kosong
def count_empty_pred_answer(dataset):
    filtered_dataset = dataset.filter(lambda x: x["pred_answer"] == "")
    return len(filtered_dataset)

# Hitung jumlah row pada split train
num_empty_pred_answer_train = count_empty_pred_answer(abstract_nq["train"])

# Cetak hasil
print(f"Jumlah row dengan pred_answer kosong pada split train: {num_empty_pred_answer_train}")


Filter:   0%|          | 0/39466 [00:00<?, ? examples/s]

Jumlah row dengan pred_answer kosong pada split train: 3


In [ ]:
# Fungsi untuk memfilter row dengan pred_answer kosong
def count_empty_gen_answer(dataset):
    filtered_dataset = dataset.filter(lambda x: x["generated_answers"] is None)
    return len(filtered_dataset)

# Hitung jumlah row pada split train
num_empty_gen_answer_train = count_empty_gen_answer(abstract_nq["train"])

# Cetak hasil
print(f"Jumlah row dengan pred_answer kosong pada split train: {num_empty_gen_answer_train}")

##### Mencari kasus dimana generated sama dengan no_retrieval tetapi keduanya berbeda dengan gold_answers

In [46]:
import re
import ast

def remove_stopword(text):
    # Hapus kata 'the' dengan batas kata
    cleaned_text = re.sub(r'\b(the|a|an)\b', '', text)  # Hapus kata 'the' dan 'a'
    cleaned_text = re.sub(r"[.,()^!?:'*+%~-]", '', cleaned_text)  # Hapus tanda titik dan koma
    # Hapus spasi ekstra yang mungkin tersisa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def fix_gold(text):
    result = text.lower().replace("\\xa0", " ").replace("\"", "")
    result = ast.literal_eval(result)
    for i, answer in enumerate(result):
        result[i] = remove_stopword(answer)
    return result

gold = fix_gold("['\"Out of Time\"']")
generated = remove_stopword("\"Out of Time\"".lower().replace("\"", ""))
print("gold:", gold[0])
print("generated:", generated)

if generated not in gold:
    print("Tidak terdapat di dalam gold answers list")

gold: out of time
generated: out of time


In [47]:
import json
import ast

# Fungsi untuk memfilter kasus unik
def filter_cases(dataset):
    filtered_rows = []
    exceptional = []
    for row in dataset:
        try:
            gold_answers = row["gold_answers"].lower().replace("\\xa0", " ").replace("\"", "")
            gold_answers_list = ast.literal_eval(gold_answers)
            for i, answer in enumerate(gold_answers_list):
                gold_answers_list[i] = remove_stopword(answer)
            generated = remove_stopword(row["generated_answers"].lower().replace("\"", ""))
            no_retrieval = remove_stopword(row["no_retrieval_pred_answer"].lower().replace("\"", ""))

            if generated != no_retrieval and generated not in gold_answers_list and no_retrieval not in gold_answers_list:
                filtered_rows.append(row)

        except Exception as e:
            # Opsional: Cetak error untuk debugging
            # print(f"Error processing row: {e}")
            exceptional.append(row)
            # print("gold:", row["gold_answers"], "--", "generated:", row["generated_answers"])
            continue  # Lewati row jika terjadi error

    return filtered_rows, exceptional

# Simpan hasil ke file JSON
def save_cases_to_json(cases, output_path):
    with open(output_path, "w") as f:
        json.dump(cases, f, indent=4)
    print(f"Hasil disimpan di: {output_path}")

# Filter kasus pada data training
sampled_cases_train, exceptional = filter_cases(abstract_nq["train"])

# Simpan hasil ke file JSON
save_cases_to_json(sampled_cases_train, "all_different.json")


Hasil disimpan di: all_different.json


Dari data tersebut, dicek apakah ada kasus dimana summary-nya kosong

In [10]:
import json

# Nama file yang akan dibaca
input_file = "inspect_gen_equal_to_no-retriev_but_different_from_gold.json"

# Load data dari file JSON
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Filter baris di mana summary hanya berisi string kosong atau whitespace
filtered_rows = [row for row in data if row["summary"].strip() == " "]

# Nama file output
output_file = "gen_eq_no-retriev_yet_empty_summary.json"

# Simpan hasil filter ke file JSON baru
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(filtered_rows, f, indent=4)

print(f"Hasil disimpan di {output_file}. Total baris: {len(filtered_rows)}")

Hasil disimpan di gen_eq_no-retriev_yet_empty_summary.json. Total baris: 0


##### Memahami nilai unik dari kolom type

In [48]:
# Ambil split train
train_data = abstract_nq["train"]

# Ambil semua nilai di kolom 'type'
type_values = [row["type"] for row in train_data]

# Dapatkan nilai unik
unique_types = set(type_values)

# Cetak hasil
print("Nilai unik di kolom 'type':", unique_types)

Nilai unik di kolom 'type': {'[good]'}


__Daftar Pertanyaan__
1. has_gold_answer_in_prompt = true itu apakah berarti gold answer terdapat di dalam passages atau di dalam summary?
`jawab: di dalam passages. sebenarnya kolom ini tidaklah relevan, karena kan yg menentukan apakah summary dibuat atau tidak itu adalah log probs nya.`
2. generated_answers itu adalah hasil dari model mendasarkan jawaban pada summary kan? Apa bedanya summary dengan gpt3_completions?
`Beda antara summary adalah gpt3_completions yang telah ditentukan apakah membantu dalam generasi jawaban atau tidak. Hal ini berarti summary antara bernilai sama persis dengan gpt3_completions atau bernilai kosong`
3. apakah gold_answer_score adalah log probs dari gold_answer di dalam gpt3_completions? 
`perlu dipahami dulu bahwa gpt3_completions bukanlah jawaban akhir, melainkan nama lain bagi summary (kurang lebih), generated_answer adalah jawaban akhir setelah menggunakan summary sebagai konteks dan score di sini adalah log probs dari generated_anwer`
4. Kalau gitu kenapa ada contoh dimana generated_answer tidak sama dengan gold_answers, tetapi gold_answer_score-nya memiliki nilai? berhubung penentuan apakah summary memiliki nilai atau justru kosong didasarkan pada skor jawaban (generated) yang benar (sesuai dgn gold), bukankah score harusnya kosong?
5. Kasus (row) mana saja yang generated_answer tidak sama dengan gold_answer? apakah max_gold
Pada tahap training kan data disusun dengan melihat apakah (1) model menjawab dengan benar (2) log probs dari jawaban natural lebih tinggi drpd jawaban 

jika summary tidak memberikan skor lebih tinggi daripada tanpa-summary (hanya mengandalkan memori parameter model), maka summary dibuat kosong. 
